# Preprocesado de datos para clustering

In [50]:
import pandas as pd
import numpy as np

In [51]:
__INTERIM_DATA_PATH__ = "../data/interim"
__PREPROCESSED_DATA_PATH__ = "../data/preprocessed"

## Import data

In [52]:
df = pd.read_csv(f"{__INTERIM_DATA_PATH__}/easy_money_no_null_df.csv")
df.drop("Unnamed: 0", axis=1, inplace=True)
df.shape

(5962863, 27)

In [53]:
df.sort_values(["pk_cid", "pk_partition"])[df["pk_cid"] == 23434]

C:\Users\alfre\AppData\Local\Temp/ipykernel_14404/1602397549.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.sort_values(["pk_cid", "pk_partition"])[df["pk_cid"] == 23434]


,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,country_id,region_code,gender,age,...,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
237919,23434,2018-01-28,2017-11-26,007,1,01 - TOP,ES,28,female,71,...,1,0,0,0,0,0,0,0,0,0
241142,23434,2018-02-28,2017-11-26,007,1,01 - TOP,ES,28,female,71,...,1,0,0,0,0,0,0,0,0,0
586764,23434,2018-03-28,2017-11-26,007,1,01 - TOP,ES,28,female,71,...,1,0,0,0,0,0,0,0,0,0
728949,23434,2018-04-28,2017-11-26,007,1,01 - TOP,ES,28,female,71,...,1,0,0,0,0,0,0,0,0,0
1222814,23434,2018-05-28,2017-11-26,007,1,01 - TOP,ES,28,female,71,...,1,0,0,0,0,0,0,0,0,0
1474866,23434,2018-06-28,2017-11-26,007,1,01 - TOP,ES,28,female,71,...,1,0,0,0,0,0,0,0,0,0
1479485,23434,2018-07-28,2017-11-26,007,1,01 - TOP,ES,28,female,71,...,1,0,0,0,0,0,0,0,0,0
2168042,23434,2018-08-28,2017-11-26,007,1,01 - TOP,ES,28,female,71,...,1,0,0,0,0,0,0,0,0,0
2171410,23434,2018-09-28,2017-11-26,007,1,01 - TOP,ES,28,female,71,...,1,0,0,0,0,0,0,0,0,0
2943589,23434,2018-10-28,2017-11-26,007,1,01 - TOP,ES,28,female,71,...,1,0,0,0,0,0,0,0,0,0


In [54]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
pk_cid,5962863.0,NaN,NaN,NaN,1234929.714088,162302.567746,15891.0,1112532.0,1231096.0,1352339.0,1553689.0
pk_partition,5962863,17,2019-05-28,442995,NaN,NaN,NaN,NaN,NaN,NaN,NaN
entry_date,5962863,1497,2017-07-28,57389,NaN,NaN,NaN,NaN,NaN,NaN,NaN
entry_channel,5962863,69,KHE,3113947,NaN,NaN,NaN,NaN,NaN,NaN,NaN
active_customer,5962863.0,NaN,NaN,NaN,0.403382,0.490576,0.0,0.0,0.0,1.0,1.0
segment,5962863,4,03 - UNIVERSITARIO,3900157,NaN,NaN,NaN,NaN,NaN,NaN,NaN
country_id,5962863,41,ES,5960611,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region_code,5962863.0,NaN,NaN,NaN,25.80418,13.603108,-1.0,13.0,28.0,36.0,52.0
gender,5962863,3,female,3087480,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age,5962863.0,NaN,NaN,NaN,29.760324,11.980339,2.0,22.0,25.0,34.0,105.0


In [55]:
gb_df = df.groupby("pk_cid")

In [56]:
def last_value(serie: pd.Series):
    return serie.iloc[-1]

In [57]:
clients_df = gb_df.agg(
    # active_customer
    active_customer_mean=("active_customer", np.mean),
    active_customer_last=("active_customer", last_value),

    # salary
    salary_mean=("salary", np.mean),
    salary_last=("salary", last_value),

    # age
    age_mean=("age", np.mean),
    age_last=("age", last_value)
)

print(clients_df.shape)
clients_df

(456373, 6)


,active_customer_mean,active_customer_last,salary_mean,salary_last,age_mean,age_last
pk_cid,,,,,,
15891,0.500000,0,-1.00,-1.00,59.000000,59
16063,0.714286,0,-1.00,-1.00,62.000000,62
16203,0.833333,1,-1.00,-1.00,70.000000,70
16502,0.888889,1,-1.00,-1.00,57.888889,58
17457,1.000000,1,102405.75,102405.75,53.117647,54
...,...,...,...,...,...,...
1553685,0.000000,0,-1.00,-1.00,52.000000,52
1553686,0.000000,0,-1.00,-1.00,30.000000,30
1553687,0.000000,0,-1.00,-1.00,21.000000,21


In [60]:
_gb_unique_country_id = gb_df["country_id"].nunique()
print(f"Hay {len(_gb_unique_country_id[_gb_unique_country_id > 1])} clientes que estan en mas de un pais")

In [59]:
clients_df.to_csv(f"{__PREPROCESSED_DATA_PATH__}/clients_df.csv")